# Precalculating AOI information
The site will show precalculated information for: WWF ecorregions, GADM admin 1 and 2 and WDPA. The minimum size of the features to be able to provide biodiversity data must be over 1000km^2. 
- Ecorregions polygon used ([arcgis online item](https://eowilson.maps.arcgis.com/home/item.html?id=be0308e81950409b919940bd03f2cae6))
- GADM polygons, level 0 and 1 (downloaded from [here](https://gadm.org/download_world.html))
- WDPA raster (june version, [provided by MOL](https://eowilson.maps.arcgis.com/home/content.html?user=MoL_HEdata&view=table&sortOrder=desc&sortField=modified&folder=1c7e82b763e54da29644eef4e048b2df#content))

**NOTE**: the polygons used to extract the information are not the same geometries shown on the app. In the app the geometries have been generalised to increase performance.


## Information to be gathered
- `population_sum`
- `land_cover_majority`
- `climate_regime_majority`
- `urban_percent`
- `rainfed_percent`
- `irrigated_percent`
- `rangeland_percent`
- `number_of_mammals`
- `number_of_amphibians`
- `number_of_birds`
- `number_of_reptiles`
- `percentage_protected`
- `mammals_list`
- `amphibians_list`
- `birds_list`
- `reptiles_list`

### Ecoregions

#### Population

In [ ]:
arcpy.sa.ZonalStatisticsAsTable("Ecoregions2017_20210825", 
                                "eco_id_int", 
                                "population2020.crf", 
                                r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\ecoregion_population_sum", 
                                "DATA", "SUM", "CURRENT_SLICE", 90)

In [ ]:
arcpy.management.JoinField("Ecoregions2017_20210825", "eco_id_int", "ecoregion_population_sum", "eco_id_int", "SUM")

In [ ]:
arcpy.management.AlterField("Ecoregions2017_20210825", "SUM", 
                            "population_sum", "population_sum", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")

#### climate regime and land cover

In [ ]:
arcpy.sa.ZonalStatisticsAsTable("Ecoregions2017_20210825", 
                                "eco_id_int", "ELU.crf", 
                                r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\ecoregion_elu_majority", 
                                "DATA", "MAJORITY", "CURRENT_SLICE", [90])

In [ ]:
arcpy.management.JoinField("Ecoregions2017_20210825", "eco_id_int", "ecoregion_elu_majority", "eco_id_int", "MAJORITY")

In [ ]:
arcpy.management.AlterField("Ecoregions2017_20210825", "MAJORITY", "majority_lc_cr", "majority land cover climate regime", 
                            "LONG", 4, "NULLABLE", "DO_NOT_CLEAR")

In [ ]:
arcpy.management.JoinField("Ecoregions2017_20210825", 
                           "majority_lc_cr", 
                           r"C:\Users\Viz1\Documents\ArcGIS\Projects\mammal_data_cube\elu_categories_lookup.csv", 
                           "elu_code", "lc_type;cr_type")

In [ ]:
arcpy.management.AlterField("Ecoregions2017_20210825", "lc_type", "land_cover_majority", "land_cover_majority", "TEXT", 8000, "NULLABLE", "DO_NOT_CLEAR")

In [ ]:
arcpy.management.AlterField("Ecoregions2017_20210825", "cr_type", "climate_regime_majority", "climate_regime_majority", "TEXT", 8000, "NULLABLE", "DO_NOT_CLEAR")

#### Protected percentage


In [4]:
arcpy.sa.ZonalStatisticsAsTable("Ecoregions2017_20210825_PolygonToRaster", "Value", "wdpa_oecm_zeros.crf", 
                                r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\wdpa_ecoregions_mean", 
                                "DATA", "MEAN", "CURRENT_SLICE", 90)

<geoprocessing server result object object at 0x0000029825526E68>

In [5]:
arcpy.management.JoinField("Ecoregions2017_20210825", "eco_id_int", 
                           "wdpa_ecoregions_mean", "Value", "MEAN")

<Result 'Ecoregions2017_20210825'>

In [ ]:
arcpy.management.AlterField("Ecoregions2017_20210825", "MEAN", "percentage_protected", "percentage_protected", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")

#### Land encroachment
look up table:
- 1: irrigated
- 2: rainfed
- 3: rangeland
- 4: urban

In [ ]:

arcpy.conversion.PolygonToRaster("Ecoregions2017_20210825", "eco_id_int", r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\Ecoregions2017_20210825_PolygonToRaster", "CELL_CENTER", "NONE", r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\land_encroachment_Subset.crf")

In [ ]:
arcpy.sa.ZonalStatisticsAsTable("Ecoregions2017_20210825", "eco_id_int", "land_encroachment.crf", 
                                r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\land_encroachment_ecoregions_all", 
                                "DATA", "ALL", "ALL_SLICES", [90])

In [ ]:
# join field from the raster
arcpy.management.JoinField("land_encroachment_ecoregions_all", "eco_id_int", 
                           "Ecoregions2017_20210825_PolygonToRaster", "Value", "Count")



In [21]:
# create new field and calculate the percent within land_encroachment_ecoregions_all,
arcpy.management.CalculateField("land_encroachment_ecoregions_all", "percent_encroachment", 
                                "round((!COUNT! / !Count_1!)*100,2)", "PYTHON3", '', "TEXT")

<Result 'land_encroachment_ecoregions_all'>

In [32]:
# select the rows for each kind of encroachment. join the field and change the name
# 1 irrigated
val = 1
slice_name = "irrigated"
arcpy.analysis.TableSelect("land_encroachment_ecoregions_all", f"C:/Users/Viz1/Documents/ArcGIS/Projects/SuperSample/SuperSample.gdb/land_{slice_name}_ecoregions", f"SliceNumber = {val}")
arcpy.management.JoinField(f"Ecoregions2017_20210825", "eco_id_int", f"land_{slice_name}_ecoregions", "eco_id_int", "percent_encroachment")
arcpy.management.AlterField("Ecoregions2017_20210825", "percent_encroachment", f"percent_{slice_name}", f"percent_{slice_name}", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")

<Result 'Ecoregions2017_20210825'>

In [33]:
val = 2
slice_name = "rainfed"
arcpy.analysis.TableSelect("land_encroachment_ecoregions_all", f"C:/Users/Viz1/Documents/ArcGIS/Projects/SuperSample/SuperSample.gdb/land_{slice_name}_ecoregions", f"SliceNumber = {val}")
arcpy.management.JoinField(f"Ecoregions2017_20210825", "eco_id_int", f"land_{slice_name}_ecoregions", "eco_id_int", "percent_encroachment")
arcpy.management.AlterField("Ecoregions2017_20210825", "percent_encroachment", f"percent_{slice_name}", f"percent_{slice_name}", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")

<Result 'Ecoregions2017_20210825'>

In [34]:
# select the rows for each kind of encroachment. join the field and change the name

val = 3
slice_name = "rangeland"
arcpy.analysis.TableSelect("land_encroachment_ecoregions_all", f"C:/Users/Viz1/Documents/ArcGIS/Projects/SuperSample/SuperSample.gdb/land_{slice_name}_ecoregions", f"SliceNumber = {val}")
arcpy.management.JoinField(f"Ecoregions2017_20210825", "eco_id_int", f"land_{slice_name}_ecoregions", "eco_id_int", "percent_encroachment")
arcpy.management.AlterField("Ecoregions2017_20210825", "percent_encroachment", f"percent_{slice_name}", f"percent_{slice_name}", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")

<Result 'Ecoregions2017_20210825'>

In [35]:
# select the rows for each kind of encroachment. join the field and change the name

val = 4
slice_name = "urban"
arcpy.analysis.TableSelect("land_encroachment_ecoregions_all", f"C:/Users/Viz1/Documents/ArcGIS/Projects/SuperSample/SuperSample.gdb/land_{slice_name}_ecoregions", f"SliceNumber = {val}")
arcpy.management.JoinField(f"Ecoregions2017_20210825", "eco_id_int", f"land_{slice_name}_ecoregions", "eco_id_int", "percent_encroachment")
arcpy.management.AlterField("Ecoregions2017_20210825", "percent_encroachment", f"percent_{slice_name}", f"percent_{slice_name}", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")

<Result 'Ecoregions2017_20210825'>

## GADM
Using non generalised polygons to extract the information
- population
- percentage protected (June version, MOL data)
- ecosystem majority

In [ ]:
arcpy.management.Merge("gadm_level1;gadm_level0", 
                       r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\gadm_level0_and_1_Merge", 
                       'GID_0 "GID_0" true true false 2147483647 Text 0 0,First,#,gadm_level1,GID_0,0,2147483647,gadm_level0,GID_0,0,2147483647;NAME_0 "NAME_0" true true false 2147483647 Text 0 0,First,#,gadm_level1,NAME_0,0,2147483647,gadm_level0,NAME_0,0,2147483647;GID_1 "GID_1" true true false 2147483647 Text 0 0,First,#,gadm_level1,GID_1,0,2147483647;NAME_1 "NAME_1" true true false 2147483647 Text 0 0,First,#,gadm_level1,NAME_1,0,2147483647;VARNAME_1 "VARNAME_1" true true false 2147483647 Text 0 0,First,#,gadm_level1,VARNAME_1,0,2147483647;NL_NAME_1 "NL_NAME_1" true true false 2147483647 Text 0 0,First,#,gadm_level1,NL_NAME_1,0,2147483647;TYPE_1 "TYPE_1" true true false 2147483647 Text 0 0,First,#,gadm_level1,TYPE_1,0,2147483647;ENGTYPE_1 "ENGTYPE_1" true true false 2147483647 Text 0 0,First,#,gadm_level1,ENGTYPE_1,0,2147483647;CC_1 "CC_1" true true false 2147483647 Text 0 0,First,#,gadm_level1,CC_1,0,2147483647;HASC_1 "HASC_1" true true false 2147483647 Text 0 0,First,#,gadm_level1,HASC_1,0,2147483647;geom_Length "geom_Length" false true true 8 Double 0 0,First,#,gadm_level1,geom_Length,-1,-1,gadm_level0,geom_Length,-1,-1;geom_Area "geom_Area" false true true 8 Double 0 0,First,#,gadm_level1,geom_Area,-1,-1,gadm_level0,geom_Area,-1,-1;country_size "country_size" true true false 4 Long 0 0,First,#,gadm_level1,country_size,-1,-1,gadm_level0,country_size,-1,-1;country_zoom "country_zoom" true true false 4 Long 0 0,First,#,gadm_level0,country_zoom,-1,-1', "NO_SOURCE_INFO")

In [7]:
arcpy.sa.ZonalStatisticsAsTable("gadm_level0_and_1_Merge", 
                                "GID_0_1", 
                                "population2020.crf", 
                                r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\gadm_population_sum", 
                                "DATA", "SUM", "CURRENT_SLICE", 90)

<geoprocessing server result object object at 0x0000029C3CE3DB20>

In [8]:
arcpy.management.JoinField("gadm_level0_and_1_Merge", 
                           "GID_0_1", 
                           "gadm_population_sum", 
                           "GID_0_1", "SUM")

<Result 'gadm_level0_and_1_Merge'>

In [9]:
arcpy.management.AlterField("gadm_level0_and_1_Merge", "SUM", 
                            "population_sum", "population_sum", 
                            "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")

<Result 'gadm_level0_and_1_Merge'>

#### climate regime and land cover

In [10]:
arcpy.sa.ZonalStatisticsAsTable("gadm_level0_and_1_Merge", 
                                "GID_0_1", "ELU.crf", 
                                r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\gadm_elu_majority", 
                                "DATA", "MAJORITY", "CURRENT_SLICE", [90])

<geoprocessing server result object object at 0x0000029C3CE3D620>

In [11]:
arcpy.management.JoinField("gadm_level0_and_1_Merge", 
                           "GID_0_1", "gadm_elu_majority", "GID_0_1", "MAJORITY")

<Result 'gadm_level0_and_1_Merge'>

In [12]:
arcpy.management.AlterField("gadm_level0_and_1_Merge", "MAJORITY", "majority_lc_cr", "majority land cover climate regime", 
                            "LONG", 4, "NULLABLE", "DO_NOT_CLEAR")

<Result 'gadm_level0_and_1_Merge'>

In [13]:
arcpy.management.JoinField("gadm_level0_and_1_Merge", 
                           "majority_lc_cr", 
                           r"C:\Users\Viz1\Documents\ArcGIS\Projects\mammal_data_cube\elu_categories_lookup.csv", 
                           "elu_code", "lc_type;cr_type")

<Result 'gadm_level0_and_1_Merge'>

In [14]:
arcpy.management.AlterField("gadm_level0_and_1_Merge", "lc_type", "land_cover_majority", "land_cover_majority", "TEXT", 8000, "NULLABLE", "DO_NOT_CLEAR")

<Result 'gadm_level0_and_1_Merge'>

In [15]:
arcpy.management.AlterField("gadm_level0_and_1_Merge", "cr_type", "climate_regime_majority", "climate_regime_majority", "TEXT", 8000, "NULLABLE", "DO_NOT_CLEAR")

<Result 'gadm_level0_and_1_Merge'>

#### Protected percentage


In [18]:
arcpy.sa.ZonalStatisticsAsTable("gadm_level0_and_1_Merge", "GID_0_1", "wdpa_oecm_zeros.crf", 
                                r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\wdpa_gadm_mean", "DATA", "MEAN", "CURRENT_SLICE", 90)

<geoprocessing server result object object at 0x0000029802FCA530>

In [19]:
arcpy.management.JoinField("gadm_level0_and_1_Merge", "GID_0_1", 
                           "wdpa_gadm_mean", "GID_0_1", "MEAN")

<Result 'gadm_level0_and_1_Merge'>

In [20]:
arcpy.management.AlterField("gadm_level0_and_1_Merge", "MEAN", "percentage_protected", "percentage_protected", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")

<Result 'gadm_level0_and_1_Merge'>

#### land encroachment

In [ ]:
arcpy.sa.ZonalStatisticsAsTable("gadm_level0_and_1_Merge", "GID_0_1", "land_encroachment.crf", 
                                r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\land_encroachment_gadm_all", 
                                "DATA", "ALL", "ALL_SLICES", 90)

In [37]:
arcpy.conversion.PolygonToRaster("gadm_level0_and_1_Merge", "GID_0_1", 
                                 r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\gadm_level_0_and_1_PolygonToRaster", "CELL_CENTER", "NONE", 
                                 r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\land_encroachment_Subset.crf")

<Result 'C:\\Users\\Viz1\\Documents\\ArcGIS\\Projects\\SuperSample\\SuperSample.gdb\\gadm_level_0_and_1_PolygonToRaster'>

In [1]:
# join field from the raster
arcpy.management.JoinField("land_encroachment_gadm_all", "GID_0_1", 
                           "gadm_level_0_and_1_PolygonToRaster", "GID_0_1", "Count")



<Result 'land_encroachment_gadm_all'>

In [2]:
# create new field and calculate the percent within land_encroachment_ecoregions_all,
arcpy.management.CalculateField("land_encroachment_gadm_all", "percent_encroachment", 
                                "round((!COUNT! / !Count_1!)*100,2)", "PYTHON3", '', "DOUBLE")

<Result 'land_encroachment_gadm_all'>

In [3]:
val = 1
slice_name = "irrigated"
arcpy.analysis.TableSelect("land_encroachment_gadm_all", f"C:/Users/Viz1/Documents/ArcGIS/Projects/SuperSample/SuperSample.gdb/land_{slice_name}_gadm01", f"SliceNumber = {val}")
arcpy.management.JoinField(f"gadm_level0_and_1_Merge", "GID_0_1", f"land_{slice_name}_gadm01", "GID_0_1", "percent_encroachment")
arcpy.management.AlterField("gadm_level0_and_1_Merge", "percent_encroachment", f"percent_{slice_name}", f"percent_{slice_name}", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")

<Result 'gadm_level0_and_1_Merge'>

In [4]:
val = 2
slice_name = "rainfed"
arcpy.analysis.TableSelect("land_encroachment_gadm_all", f"C:/Users/Viz1/Documents/ArcGIS/Projects/SuperSample/SuperSample.gdb/land_{slice_name}_gadm01", f"SliceNumber = {val}")
arcpy.management.JoinField(f"gadm_level0_and_1_Merge", "GID_0_1", f"land_{slice_name}_gadm01", "GID_0_1", "percent_encroachment")
arcpy.management.AlterField("gadm_level0_and_1_Merge", "percent_encroachment", f"percent_{slice_name}", f"percent_{slice_name}", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")

<Result 'gadm_level0_and_1_Merge'>

In [5]:
val = 3
slice_name = "rangeland"
arcpy.analysis.TableSelect("land_encroachment_gadm_all", f"C:/Users/Viz1/Documents/ArcGIS/Projects/SuperSample/SuperSample.gdb/land_{slice_name}_gadm01", f"SliceNumber = {val}")
arcpy.management.JoinField(f"gadm_level0_and_1_Merge", "GID_0_1", f"land_{slice_name}_gadm01", "GID_0_1", "percent_encroachment")
arcpy.management.AlterField("gadm_level0_and_1_Merge", "percent_encroachment", f"percent_{slice_name}", f"percent_{slice_name}", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")

<Result 'gadm_level0_and_1_Merge'>

In [6]:
val = 4
slice_name = "urban"
arcpy.analysis.TableSelect("land_encroachment_gadm_all", f"C:/Users/Viz1/Documents/ArcGIS/Projects/SuperSample/SuperSample.gdb/land_{slice_name}_gadm01", f"SliceNumber = {val}")
arcpy.management.JoinField(f"gadm_level0_and_1_Merge", "GID_0_1", f"land_{slice_name}_gadm01", "GID_0_1", "percent_encroachment")
arcpy.management.AlterField("gadm_level0_and_1_Merge", "percent_encroachment", f"percent_{slice_name}", f"percent_{slice_name}", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")

<Result 'gadm_level0_and_1_Merge'>

## WDPA
Using non generalised polygons to extract the information
- population
- percentage protected (June version, MOL data)
- ecosystem majority